# work on the tree

## ast.NodeVisitor

ast.NodeVisitor is the primary tool for ‘scanning’ the tree. 

In [2]:
import ast
import inspect

print(inspect.getsource(ast.NodeVisitor))

class NodeVisitor(object):
    """
    A node visitor base class that walks the abstract syntax tree and calls a
    visitor function for every node found.  This function may return a value
    which is forwarded by the `visit` method.

    This class is meant to be subclassed, with the subclass adding visitor
    methods.

    Per default the visitor functions for the nodes are ``'visit_'`` +
    class name of the node.  So a `TryFinally` node visit function would
    be `visit_TryFinally`.  This behavior can be changed by overriding
    the `visit` method.  If no visitor function exists for a node
    (return value `None`) the `generic_visit` visitor is used instead.

    Don't use the `NodeVisitor` if you want to apply changes to nodes during
    traversing.  For this a special visitor exists (`NodeTransformer`) that
    allows modifications.
    """

    def visit(self, node):
        """Visit a node."""
        method = 'visit_' + node.__class__.__name__
        visitor = getattr(

To use it, subclass it and override methods visit_Foo, corresponding to the node classes. (see [Meet the Nodes](https://greentreesnakes.readthedocs.io/en/latest/nodes.html)).

For example, this visitor will print the names of any functions defined in the given code, including methods and functions defined within other functions:

In [11]:
import ast
class FuncLister(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print('func_name: ', node.name)
        self.generic_visit(node)

In [13]:
source_code = """
def a():
    print('i am a')
    
def b():
    print('call a')
    a()
    def c():
        print('i am c function')
    
b()
""".strip()
FuncLister().visit(ast.parse(source_code))

func_name:  a
func_name:  b
func_name:  c


If you want child nodes to be visited, remember to call self.generic_visit(node) in the methods you override.

Alternatively, you can run through a list of all the nodes in the tree using ast.walk(). There are no guarantees about the order in which nodes will appear. The following example again prints the names of any functions defined within the given code:

In [14]:
import ast

source_code = """
def a():
    print('i am a')
    
def b():
    print('call a')
    a()
    def c():
        print('i am c function')
    
b()
""".strip()

tree = ast.parse(source_code)
for node in ast.walk(tree):
    if isinstance(node, ast.FunctionDef):
        print('func_name: ', node.name)

func_name:  a
func_name:  b
func_name:  c
